In [1]:
import numpy as np
import pickle as pkl
import os
from PIL import Image
from matplotlib import pyplot as plt
import matplotlib.patches as patches
from math import floor, ceil
import time
from collections import deque
import random
from IPython.display import clear_output

In [2]:
feature_before = np.random.rand(50,70,1024)
feature_after = np.random.rand(50,70,1024)

In [3]:
def max_pool_dynamic_input(x, H_out, W_out):
    out = None
    (N, H, W, C) = x.shape
    pool_height = np.zeros((H_out), dtype="int32")
    pool_width = np.zeros((W_out), dtype="int32")
    
    div_h = int(H/H_out)
    mod_h = H%H_out
    div_w = int(W/W_out)
    mod_w = W%W_out
    pool_height = pool_height + div_h
    pool_width = pool_width + div_w
    pool_height[:mod_h] += 1
    pool_width[:mod_w] += 1
    
    out = np.zeros((N, H_out, W_out, C))
    for n in range(N):
        for h in range(H_out):
            for w in range(W_out):
                h1 = sum(pool_height[:h])
                h2 = h1 + pool_height[h]
                w1 = sum(pool_width[:w])
                w2 = w1 + pool_width[w]
                window = x[n, h1:h2, w1:w2, :]
                m = np.max(window, axis=0)
                m = np.max(m, axis=0)
                out[n,h,w,:] = m
    
    return out

In [4]:
def get_max_and_concat(*args):
    size = 0
    for arg in args:
        size += arg.shape[-1]
    final = np.zeros(size)
    
    arg_size = 0
    
    i = 0
    for arg in args:
        max_pool = max_pool_dynamic_input(np.expand_dims(arg, 0), 1, 1)[0,0, 0,:]
        final[i: arg.shape[-1]+i] = max_pool
        i+= arg.shape[-1]  
    return final
        

In [18]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou

In [19]:
def filter_detections(detections, detection_labels, max_iou=.3, max_filtered=10, rand_select=2):
    filtered_idx = []
    for i in range(len(detections)):
        detection = detections[i]
        add_to_filter = True
        for label in detection_labels:
            if bb_intersection_over_union(detection, label) > max_iou:
                add_to_filter = False
                break

        for fi in filtered_idx:
            if bb_intersection_over_union(detection, detections[fi]) > 0.1:
                add_to_filter = False
                break

        if add_to_filter:
            filtered_idx.append(i)

        if len(filtered_idx) > max_filtered:
            break
    if len(filtered_idx) > rand_select:
        filtered_idx = random.sample(filtered_idx, rand_select)
    
    return detections[filtered_idx]

In [20]:
SESSIONS_PATH = "/home/wc-gpu/storage4tb/session_data_thesis/sessions160000_165000"
IMAGE_SIZE = (30, 20)
plot_features_and_annotations = True

In [ ]:
import random

for session in sorted(os.listdir(SESSIONS_PATH)):
    try:
        a = int(session)
        session_path = os.path.join(SESSIONS_PATH, session)
        ann_path = os.path.join(session_path, "boxes")
        features_path = os.path.join(session_path, "features")
        
        if os.path.exists(features_path) and os.path.exists(boxes_path):
            actual_sessions.append(session)
            images = [f for f in os.listdir(session_path) if f.endswith(".jpg")]
            total_images += len(images)
    except:
        print (session)
        continue

In [21]:
X2all = dict()
Y2all = dict()

X3all = dict()
Y3all = dict()

total_images = 0
sessions = os.listdir(SESSIONS_PATH)
actual_sessions = []

for session in sorted(sessions, reverse=True):
    try:
        a = int(session)
        session_path = os.path.join(SESSIONS_PATH, session)
        boxes_path = os.path.join(session_path, "boxes")
        features_path = os.path.join(session_path, "features")
        
        if os.path.exists(features_path) and os.path.exists(boxes_path):
            actual_sessions.append(session)
            images = [f for f in os.listdir(session_path) if f.endswith(".jpg")]
            total_images += len(images)
    except:
        print (session)
        continue
        
progress = 0

to_save = 0
time_avg = 0

count_saved = 0
for session in sorted(actual_sessions):
    try:
        features_path = os.path.join(SESSIONS_PATH, session, "features")
        boxes_path = os.path.join(SESSIONS_PATH, session, "boxes")
        session_path = os.path.join(SESSIONS_PATH, session)
        previous_features = None
        previous_boxes = None
        prev2_features = None
        prev2_boxes = None
        X2 = []
        Y2 = []
        X3 = []
        Y3 = []
        for image_file in sorted(os.listdir(session_path)):
            try:
                if image_file.endswith(".jpg"):
                    
                    t0 = time.time()

                    feature_path = os.path.join(features_path, image_file + ".feature.prod.new.better")
                    if not os.path.exists(features_path):
                        break
                    boxe_path = os.path.join(boxes_path, image_file + ".box.better.not_filtered")
                    features = None
                    boxes = None
                    with open(feature_path, "rb") as f:
                        features = np.load(f)[0]
                    with open(boxe_path, "rb") as f:
                        boxes = pkl.load(f)
                    
                    
                    print (features.shape)
                    detection_boxes = boxes[0]
                    detection_scores = boxes[1]
                    annotations = boxes[2]
                    
                    detection_boxes = filter_detections(detection_boxes, annotations)
                    boxes = (detection_boxes, annotations)
                    print (len(detection_boxes), len(annotations))

                    if previous_features is not None:
                        for label in boxes[0]:
                            box_prev = previous_features[
                                floor(label[0]*previous_features.shape[0]):
                                ceil(label[2]*previous_features.shape[0]), 
                                floor(label[1]*previous_features.shape[1]):
                                ceil(label[3]*previous_features.shape[1]),:]
                            box_cur = features[
                                floor(label[0]*features.shape[0]):
                                ceil(label[2]*features.shape[0]), 
                                floor(label[1]*features.shape[1]):
                                ceil(label[3]*features.shape[1]),:]
                            
                            max_pooled = get_max_and_concat(box_prev, previous_features, box_cur, features)
                            X2.append(max_pooled)
                            Y2.append(0)

                            if prev2_features is not None:
                                box_prev2 = prev2_features[
                                    floor(label[0]*previous_features.shape[0]):
                                    ceil(label[2]*previous_features.shape[0]), 
                                    floor(label[1]*previous_features.shape[1]):
                                    ceil(label[3]*previous_features.shape[1]),:]
                                max_pooled = get_max_and_concat(
                                    box_prev2, prev2_features,
                                    box_prev, previous_features,
                                    box_cur, features)
                                X3.append(max_pooled)
                                Y3.append(0)

                            
                            
                        for label in boxes[1]:
                            if(label[4] == "add"):
                                box_prev = previous_features[
                                    floor(label[0]*previous_features.shape[0]):
                                    ceil(label[2]*previous_features.shape[0]), 
                                    floor(label[1]*previous_features.shape[1]):
                                    ceil(label[3]*previous_features.shape[1]),:]
                                box_cur = features[
                                    floor(label[0]*features.shape[0]):
                                    ceil(label[2]*features.shape[0]), 
                                    floor(label[1]*features.shape[1]):
                                    ceil(label[3]*features.shape[1]),:]

                                max_pooled = get_max_and_concat(box_prev, previous_features, box_cur, features)
                                X2.append(max_pooled)
                                Y2.append(1)
                                
                                if prev2_features is not None:
                                    box_prev2 = prev2_features[
                                        floor(label[0]*previous_features.shape[0]):
                                        ceil(label[2]*previous_features.shape[0]), 
                                        floor(label[1]*previous_features.shape[1]):
                                        ceil(label[3]*previous_features.shape[1]),:]
                                    
                                    max_pooled = get_max_and_concat(
                                        box_prev2, prev2_features,
                                        box_prev, previous_features,
                                        box_cur, features)
                                    X3.append(max_pooled)
                                    Y3.append(1)


                        for label in previous_boxes[1]:
                            if(label[4] == "remove"):
                                box_prev = previous_features[
                                    floor(label[0]*previous_features.shape[0]):
                                    ceil(label[2]*previous_features.shape[0]), 
                                    floor(label[1]*previous_features.shape[1]):
                                    ceil(label[3]*previous_features.shape[1]),:]
                                box_cur = features[
                                    floor(label[0]*features.shape[0]):
                                    ceil(label[2]*features.shape[0]), 
                                    floor(label[1]*features.shape[1]):
                                    ceil(label[3]*features.shape[1]),:]

                                max_pooled = get_max_and_concat(box_cur, features, box_prev, previous_features)
                                X2.append(max_pooled)
                                Y2.append(1)
                                
                        if prev2_boxes is not None:
                            for label in prev2_boxes[1]:
                                if(label[4] == "remove"):
                                    box_prev = previous_features[
                                        floor(label[0]*previous_features.shape[0]):
                                        ceil(label[2]*previous_features.shape[0]), 
                                        floor(label[1]*previous_features.shape[1]):
                                        ceil(label[3]*previous_features.shape[1]),:]
                                    box_cur = features[
                                        floor(label[0]*features.shape[0]):
                                        ceil(label[2]*features.shape[0]), 
                                        floor(label[1]*features.shape[1]):
                                        ceil(label[3]*features.shape[1]),:]

                                    box_prev2 = prev2_features[
                                        floor(label[0]*previous_features.shape[0]):
                                        ceil(label[2]*previous_features.shape[0]), 
                                        floor(label[1]*previous_features.shape[1]):
                                        ceil(label[3]*previous_features.shape[1]),:]
                                    
                                    max_pooled = get_max_and_concat(
                                        box_cur, features,
                                        box_prev, previous_features,
                                        box_prev2, prev2_features)
                                    X3.append(max_pooled)
                                    Y3.append(1)

                    prev2_features = previous_features
                    prev2_boxes = previous_boxes
                    previous_features = features
                    previous_boxes = boxes

                    progress += 1
                    to_save += 1

                    t1 = time.time()

                    total = t1-t0
                    total_time += total
                    
                    time_avg = total_time / progress
                
            except Exception as ex:
                raise ex
                print (session, ex)
        clear_output()
        print ("progress", session, "  : ", progress / total_images, progress, total_images)
        print ("time", session, "  passed: ", time_avg*progress / 3600 ," h  expected left: ", time_avg* (total_images - progress) / 3600, " h")
        
        X2 = np.asarray(X2)
        Y2 = np.asarray(Y2)
        X3 = np.asarray(X3)
        Y3 = np.asarray(Y3)
        
        print(X2.shape, Y2.shape, X3.shape, Y3.shape)
        
        
        X2all[session] = X2
        X3all[session] = X3
        Y2all[session] = Y2
        Y3all[session] = Y3
        
       

    except Exception as ex:
        raise ex

        print (session, ex)
    

progress 164413   :  1.0 25325 25325
time 164413   passed:  3.410640763207011  h  expected left:  0.0  h
(152, 4096) (152,) (150, 6144) (150,)


In [22]:
# X2 = np.asarray(X2)
# Y2 = np.asarray(Y2)
# X3 = np.asarray(X3)
# Y3 = np.asarray(Y3)

In [23]:
with open(SESSIONS_PATH + "/dataset_prod_new_model_binary_1prev_session_split.pkl", "wb") as f:
    pkl.dump([X2all, Y2all], f)

In [24]:
with open(SESSIONS_PATH + "/dataset_prod_new_model_binary_2prev_session_split.pkl", "wb") as f:
    pkl.dump([X3all, Y3all], f)

In [ ]:
# for session in sorted(os.listdir(SESSIONS_PATH)):
#     try:
#         a = int(session)

#         features_path = os.path.join(SESSIONS_PATH, session, "features")
#         boxes_path = os.path.join(SESSIONS_PATH, session, "boxes")
#         session_path = os.path.join(SESSIONS_PATH, session)
#         if not os.path.exists(os.path.join(features_path)) or not os.path.exists(os.path.join(boxes_path)) :
#             continue

#         for image_file in sorted(os.listdir(session_path)):
#             try:
#                 if not image_file.endswith(".jpg"):
#                     continue
#                 feature_path = os.path.join(features_path, image_file + ".feature.prod.new")
                
#                 boxe_path = os.path.join(boxes_path, image_file + ".box")
#                 features = None
#                 boxes = None
#                 with open(feature_path, "rb") as f:
#                     features = np.load(f)
#                 with open(boxe_path, "rb") as f:
#                     boxes = pkl.load(f)
                
#                 features = features[0]
#                 print ("SHAPE", features.shape)
#                 image_np = features[:,:,500]
#                 image_np *= 255.0/image_np.max()  

#                 fig,ax = plt.subplots(1, figsize=IMAGE_SIZE)

#                 ax.imshow(image_np)
#                 for label in boxes[0]:
#                     rect = patches.Rectangle(
#                         (label[1]*image_np.shape[1],label[0]*image_np.shape[0]),
#                         (label[3]-label[1])*image_np.shape[1],
#                         (label[2]-label[0])*image_np.shape[0],
#                         linewidth=5,edgecolor='r',facecolor='none')
#                     ax.add_patch(rect)

#                 for label in boxes[1]:
#                     rect = patches.Rectangle(
#                         (label[1]*image_np.shape[1],label[0]*image_np.shape[0]),
#                         (label[3]-label[1])*image_np.shape[1],
#                         (label[2]-label[0])*image_np.shape[0],
#                         linewidth=5,edgecolor='g',facecolor='none')
#                     ax.add_patch(rect)
#                 plt.show(IMAGE_SIZE)
                
                
#                 print("negatives")
#                 for label in boxes[0]:
# #                     print (
# #                         floor(label[0]*image_np.shape[0]),
# #                         ceil(label[2]*image_np.shape[0]),
# #                         floor(label[1]*image_np.shape[1]),
# #                         ceil(label[3]*image_np.shape[1]))
#                     rect = features[
#                         floor(label[0]*image_np.shape[0]):
#                         ceil(label[2]*image_np.shape[0]), 
#                         floor(label[1]*image_np.shape[1]):
#                         ceil(label[3]*image_np.shape[1])]
#                     fig,ax = plt.subplots(1)
#                     ax.imshow(rect[:,:,500])
#                     plt.show()
                    
                
                
#                 print("positives")
#                 for label in boxes[1]:
#                     rect = features[
#                         floor(label[0]*image_np.shape[0]):
#                         ceil(label[2]*image_np.shape[0]), 
#                         floor(label[1]*image_np.shape[1]):
#                         ceil(label[3]*image_np.shape[1])]
#                     fig,ax = plt.subplots(1)
#                     ax.imshow(rect[:,:,500])
#                     plt.show()
                    
                
#             except Exception as ex:
#                 print (session, ex)
#                 continue
#         break
#     except Exception as ex:
#         print (session, ex)
#         continue
    

In [ ]:
X = [[], [], []]
Y = [[], [], []]

total_images = 0
sessions = os.listdir(SESSIONS_PATH)
actual_sessions = []

for session in sorted(sessions, reverse=True):
    try:
        a = int(session)
        session_path = os.path.join(SESSIONS_PATH, session)
        boxes_path = os.path.join(session_path, "boxes")
        features_path = os.path.join(session_path, "features")
        
        if os.path.exists(features_path) and os.path.exists(boxes_path):
            actual_sessions.append(session)
            images = [f for f in os.listdir(session_path) if f.endswith(".jpg")]
            total_images += len(images)
    except:
        print (session)
        continue
        
progress = 0

processed = 0
time_avg = 0

for session in sorted(actual_sessions):
    try:
        features_path = os.path.join(SESSIONS_PATH, session, "features")
        boxes_path = os.path.join(SESSIONS_PATH, session, "boxes")
        session_path = os.path.join(SESSIONS_PATH, session)
        previous_features = [None, None, None, None]
        previous_boxes = [None, None, None, None]
        for image_file in sorted(os.listdir(session_path)):
            try:
                if image_file.endswith(".jpg"):

                    t0 = time.time()

                    feature_path = os.path.join(features_path, image_file + ".feature.prod.new.better")
                    if not os.path.exists(features_path):
                        break
                    boxe_path = os.path.join(boxes_path, image_file + ".box.better.not_filtered")
                    features = None
                    boxes = None
                    with open(feature_path, "rb") as f:
                        features = np.load(f)[0]
                    with open(boxe_path, "rb") as f:
                        boxes = pkl.load(f)
                    
                    
                    detetction_boxes = boxes[0]
                    detection_scores = boxes[1]
                    annotations = boxes[2]
                    
                    detection_boxes = filter_detections(detetction_boxes, annotations)
                    
                    boxes = (detetction_boxes, annotations)
                    
                    previous_features.push() features
                    previous_boxes = boxes
                    
                    Xdata = [[],[],[],[]]
                    for prev_feat_i in range(len(previous_features)):
                        prev_feat
                        if prev_feat is not None:
                            for label in boxes[0]:
                                box_prev = prev_feat[
                                    floor(label[0]*previous_features.shape[0]):
                                    ceil(label[2]*previous_features.shape[0]), 
                                    floor(label[1]*previous_features.shape[1]):
                                    ceil(label[3]*previous_features.shape[1]),:]
                                box_cur = features[
                                    floor(label[0]*features.shape[0]):
                                    ceil(label[2]*features.shape[0]), 
                                    floor(label[1]*features.shape[1]):
                                    ceil(label[3]*features.shape[1]),:]

                                max_pooled = get_max_and_concat(box_prev, prev_feat, box_cur, features)
                                X.append(max_pooled)
                                Y.append(0)

                            for label in boxes[1]:
                                if(label[4] == "add"):
                                    box_prev = previous_features[
                                        floor(label[0]*prev_feat.shape[0]):
                                        ceil(label[2]*prev_feat.shape[0]), 
                                        floor(label[1]*prev_feat.shape[1]):
                                        ceil(label[3]*prev_feat.shape[1]),:]
                                    box_cur = features[
                                        floor(label[0]*features.shape[0]):
                                        ceil(label[2]*features.shape[0]), 
                                        floor(label[1]*features.shape[1]):
                                        ceil(label[3]*features.shape[1]),:]

                                    max_pooled = get_max_and_concat(box_prev, prev_feat, box_cur, features)
                                    X.append(max_pooled)
                                    Y.append(1)

                        for label in previous_boxes[1]:
                            if(label[4] == "remove"):
                                box_prev = previous_features[
                                    floor(label[0]*previous_features.shape[0]):
                                    ceil(label[2]*previous_features.shape[0]), 
                                    floor(label[1]*previous_features.shape[1]):
                                    ceil(label[3]*previous_features.shape[1]),:]
                                box_cur = features[
                                    floor(label[0]*features.shape[0]):
                                    ceil(label[2]*features.shape[0]), 
                                    floor(label[1]*features.shape[1]):
                                    ceil(label[3]*features.shape[1]),:]

                                max_pooled = get_max_and_concat(box_prev, previous_features, box_cur, features)
                                X.append(max_pooled)
                                Y.append(1)

                    

                    total_time = time_avg * processed
                    processed += 1

                    t1 = time.time()

                    total = t1-t0
                    print ("total time", total)

                    time_avg = (total_time + total) / processed

                    print ("progress", session, "  : ", progress / total_images, progress, total_images)
                    print ("time", session, "  passed: ", time_avg*processed / 3600 ," h  expected left: ", time_avg* (total_images - progress) / 3600, " h")

                    progress += 1
                
            except Exception as ex:
                print (session, ex)
            
            
    except Exception as ex:
        print (session, ex)
    